In [ ]:
import os
import sys
import pandas as pd
pd.set_option('display.max_colwidth', None)
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from util.graphdb_base import GraphDBBase

In [ ]:
gd = GraphDBBase()
gd._database = "ned"
query = "MATCH (n) RETURN n LIMIT 10"
with gd._driver.session(database=gd._database) as session:
    r = session.run(query)
    if len(r.data()) == 0:
        raise RuntimeError("No data in the database")

In [ ]:
def show_result_in_df(query):
    with gd._driver.session(database=gd._database) as session:
        r = session.run(query)
        return pd.DataFrame.from_dict(r.data())

In [ ]:
query_9_24 = """
CALL db.index.fulltext.queryNodes("PageText", "breakbone fever")
YIELD node, score
WITH node as p, score as score
MATCH (f:File)-[:CONTAINS_PAGE]->(p)
RETURN f.id as `File ID`, p.page_idx as `Page index`, score as Score
LIMIT 5
"""
res_9_24 = show_result_in_df(query_9_24)
res_9_24

In [ ]:
query_9_25 = """
MATCH path=(f:File)-[:CONTAINS_PAGE]->(p)-[r:MENTIONS_MENTION]->(m)-[:DISAMBIGUATED_TO]->(e)
WHERE "breakbone fever" in [x in e.aliases | toLower(x)] 
UNWIND range(0, size(r.start_chars)-1) as mention 
WITH f, p, e, m, r, mention, collect(p) as pages_list
RETURN DISTINCT f.id as `File ID`, p.page_idx as `Page index`, apoc.text.join(collect(substring (p.text, apoc.coll.max([r.start_chars[mention] - 100, 0]), (r.end_chars[mention] - r.start_chars[mention] + 200) ))[0..3], '\n\n') as `Mention contexts`, size(collect(m.name)) as `Number of mentions`
ORDER BY `Number of mentions` DESC
LIMIT 5
"""
res_9_25 = show_result_in_df(query_9_25)
res_9_25

In [ ]:
query_9_26 = """
MATCH (m1:MedicalEntity)-[:IS_SNOMED_ENTITY]->(s1:SnomedEntity)<-[r1:SNOMED_RELATION]-(s2:SnomedEntity)<-[:IS_SNOMED_ENTITY]-(e:MedicalEntity)
WHERE "islets of Langerhans" in m1.aliases AND r1.type = "FINDING_SITE"
WITH e

MATCH path=(f:File)-[:CONTAINS_PAGE]->(p)-[r:MENTIONS_MENTION]->(m)-[:DISAMBIGUATED_TO]->(e)
UNWIND range(0, size(r.start_chars)-1) as mention 
WITH f, p, r, e, mention
RETURN DISTINCT f.id as `File ID`,
p.page_idx as `Page index`,
collect(distinct e.name) as `Mentioned entities`,
apoc.text.join(collect(substring (p.text, apoc.coll.max([r.start_chars[mention] - 100, 0]), (r.end_chars[mention] - r.start_chars[mention] + 200) ))[0..3], '\n\n') as `Mention contexts`
ORDER BY size(`Mentioned entities`) DESC
LIMIT 5
"""
res_9_26 = show_result_in_df(query_9_26)
res_9_26

In [ ]:
query_9_27 = """
MATCH (m1:MedicalEntity)-[:IS_SNOMED_ENTITY]->(s1:SnomedEntity)-[r1:SNOMED_RELATION*3..3]-(s2:SnomedEntity)<-[:IS_SNOMED_ENTITY]-(e:MedicalEntity)
WHERE m1.name = "Zika Virus" AND all(x IN r1 WHERE x.type = "CAUSATIVE_AGENT")
WITH DISTINCT e

MATCH path=(f:File)-[:CONTAINS_PAGE]->(p)-[r:MENTIONS_MENTION]->(m)-[:DISAMBIGUATED_TO]->(e)
WITH f, e, collect(p.page_idx) as pages_list
RETURN DISTINCT f.id as `File ID`, pages_list, collect(distinct e.name) as `Mentioned entity`
ORDER BY size(`Mentioned entity`) DESC
LIMIT 5
"""
res_9_27 = show_result_in_df(query_9_27)
res_9_27

In [ ]:
query_9_31 = """
MATCH (m1:MedicalEntity)-[r:COOCCURR]-(m2:MedicalEntity)
WHERE m1.id= "C0318793"
RETURN m2.type as `Entity Type`, count(m2.type) as `Number of co-occurrences` 
ORDER BY count(m2.type) DESC
LIMIT 5
"""
res_9_31 = show_result_in_df(query_9_31)
res_9_31

In [ ]:
query_9_32 = """
MATCH (n1:MedicalEntity)-[r:COOCCURR]-(n2:MedicalEntity)
WHERE n1.id= "C0318793" and n2.type = "Disease or Syndrome" and n2.id <> "C0009450"
WITH n1, r, n2
ORDER BY r.count DESC

MATCH (f:File)-[:CONTAINS_PAGE]->(p:Page)-[r1:MENTIONS_MENTION]->(m1)-[:DISAMBIGUATED_TO]->(n1),
(p)-[r2:MENTIONS_MENTION]->(m2)-[:DISAMBIGUATED_TO]->(n2)
WHERE r1.sentence_index = r2.sentence_index
WITH f, p, r1, r2, n2

RETURN DISTINCT
f.id as `File ID`,
p.page_idx as `Page index`,
n2.id as `Co-occurring entity id`,
n2.name as `Co-occurring entity name`,
CASE
WHEN r1.end_chars[0] > r2.start_chars[0]
THEN substring(p.text, r2.start_chars[0], r1.end_chars[0] - r2.start_chars[0])
ELSE substring(p.text, r1.start_chars[0], r2.end_chars[0] - r1.start_chars[0])
END as `Mentions context`
"""
res_9_32 = show_result_in_df(query_9_32)
res_9_32

In [ ]:
query_9_33 = """
MATCH (s1), (s2)
WHERE s1.id="3928002" AND s2.id="50471002"
WITH s1, s2, allShortestPaths((s1)-[:SNOMED_RELATION*1..8]-(s2)) AS paths
UNWIND paths AS path
WITH relationships(path) AS path_edges,
nodes(path) as path_nodes
WITH [n IN path_nodes | n.name] AS node_names,
[r IN path_edges | r.type] AS rel_types,
[n IN path_edges | startnode(n).name] AS rel_starts
WITH [i in range(0, size(node_names)-1) | CASE
WHEN i = size(node_names)-1
THEN '(' + node_names[size(node_names)-1] + ')'
WHEN node_names[i] = rel_starts[i]
THEN '(' + node_names[i] + ')' + '-[:' + rel_types[i] + ']->'
ELSE '(' + node_names[i] + ')' + '<-[:' + rel_types[i] + ']-' END] as string_paths
RETURN DISTINCT apoc.text.join(string_paths, '') AS `Extracted paths`
"""
res_9_33 = show_result_in_df(query_9_33)
res_9_33

In [ ]:
query_9_36 = """
MATCH (d:Disease)<-[:IS_DISEASE_ENTITY]-(n1:MedicalEntity)-[r:COOCCURR]-(n2:MedicalEntity)-[:IS_HPO_ENTITY]->(h:Hpo)
WHERE EXISTS((d)-[:HAS_PHENOTYPIC_FEATURE]->(h)) and n2.id = "C0004114"
WITH n1, n2, d, h

MATCH (f:File)-[:CONTAINS_PAGE]->(p:Page)-[r1:MENTIONS_MENTION]->(m1)-[:DISAMBIGUATED_TO]->(n1),
(p)-[r2:MENTIONS_MENTION]->(m2)-[:DISAMBIGUATED_TO]->(n2)
WHERE r1.sentence_index = r2.sentence_index
WITH f, p, r1, r2, n1, n2, d, h

RETURN DISTINCT
f.id as `File name`,
p.page_idx as `Page index`,
d.label as `Disease name`,
CASE
WHEN r1.end_chars[0] > r2.start_chars[0]
THEN substring(p.text, r2.start_chars[0], r1.end_chars[0] - r2.start_chars[0])
ELSE substring(p.text, r1.start_chars[0], r2.end_chars[0] - r1.start_chars[0])
END as `Mentions context`
"""
res_9_36 = show_result_in_df(query_9_36)
res_9_36

In [ ]:
query_9_37 = """
MATCH (n1:MedicalEntity)-[r:COOCCURR]-(n2:MedicalEntity)
WHERE n1.id= "C0318793" and n2.type = "Disease or Syndrome"
RETURN n2.name, r.count
ORDER BY r.count DESC
LIMIT 5
"""
res_9_37 = show_result_in_df(query_9_37)
res_9_37

In [ ]:
query_9_39 = """
CALL gds.graph.project(
  'snomedGraph',
  'SnomedEntity',
  'SNOMED_RELATION'
)
YIELD
  graphName AS graph,
  relationshipProjection AS knowsProjection,
  nodeCount AS nodes,
  relationshipCount AS rels
"""
res_9_39 = show_result_in_df(query_9_39)
res_9_39

In [ ]:
query_9_40 = """
CALL gds.degree.stream('snomedGraph')
YIELD nodeId, score
WITH gds.util.asNode(nodeId).name AS name, score AS degree
ORDER BY degree DESC
LIMIT 350
WITH collect(name) as hub_nodes
MATCH (s1), (s2)
WHERE s1.id="3928002" AND s2.id="40956001"
WITH s1, s2, allShortestPaths((s1)-[:SNOMED_RELATION*1..8]-(s2)) AS paths, hub_nodes
UNWIND paths AS path
WITH relationships(path) AS path_edges, nodes(path) as path_nodes, hub_nodes
WITH [n IN path_nodes | n.name] AS node_names,
     [r IN path_edges | r.type] AS rel_types,
     [n IN path_edges | startnode(n).name] AS rel_starts,
     hub_nodes
WHERE not any(x IN node_names WHERE x IN hub_nodes)
WITH [i in range(0, size(node_names)-1) | CASE
WHEN i = size(node_names)-1
THEN "(" + node_names[size(node_names)-1] + ")"
WHEN node_names[i] = rel_starts[i] 
THEN "(" + node_names[i] + ")" + '-[:' + rel_types[i] + ']->'
ELSE "(" + node_names[i] + ")" + '<-[:' + rel_types[i] + ']-' END] as string_paths
RETURN DISTINCT apoc.text.join(string_paths, '') AS `Extracted paths`
"""
res_9_40 = show_result_in_df(query_9_40)
res_9_40